In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BSoup

In [ ]:
def get_Ibd_rank_and_group (ticker):
    #first, search for the ticker symbol
    url_name1 = "http://myibd.investors.com/search/searchresults.aspx?Ntt={}".format (ticker.to_lower)
    req1 = requests.get(url_name1)
    #then, find the link to the stock's page by following down the chain of links
    #not sure if there is a more direct path to the stock's page
    soup1 =  BSoup (req1.content, "lxml")
    #if not a direct hit, there is a second step (hop) necessary
    url_name2 = soup1.find ("a", {"id": "ctl00_ctl00_secondaryContent_leftContent_SearchResultsMgrCtrl_didYouMeanCompanyRepeater_ctl00_didYouMeanLink"})
    if url_name2:
        req2 = requests.get (url_name2.get('href'))
        soup2 = BSoup (req2.content, "lxml")
    #<span id="ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany" onclick="javascript:window.open('http://research.investors.com/stock-quotes/nyse-thor-industries-inc-tho.htm')">Thor Industries Inc</span>
        url_name3 = soup2.find ("span", {"id": "ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany"})
    else:
        url_name3 = soup1.find ("span", {"id": "ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany"}) 
    tokens = url_name3.get ('onclick').split ("'")
    for t in tokens:
        if "http://" in t:
            stock_link = t
            break
    req3 = requests.get (stock_link)
    soup3 = BSoup (req3.content, "lxml")
    #finally got the page. Now find the ticker's group, ranking, and no.1 in that group
    market_group = soup3.find ("div", {"class": "spansubHead"})
    #print (market_group.text)
    
    #if ticker is no.1, there should only be one anchor-tag, if not, there should be two (verify)
    ticker_ranks = soup3.find_all ("span", {"id":"ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank"})
    #print (spans_id1.text)
    
    #if ticker is not no. 1, the leader can be found in another anchor-tag inside the StockRolldiv:
    #<a class="stockRoll" href="http://research.investors.com/stock-quotes/nasdaq-alarmcom-holdings-inc-alrm.htm" rel="/stocksymbol.axd?symbol=ALRM">ALRM</a>
    #need to verify searching for this only yields one result
    group_no1 = soup3.find ("span", {"id": })

